In [1]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import pandas as pd


In [3]:
original = pd.read_pickle('original_adult.pickle')
mitigated = pd.read_pickle('mitigated_adult_r_e.pickle')

In [4]:
original_X = original[['sex', 'race', 'age','education']]
original_y = original[['label']]
mitigated_X = mitigated[['sex', 'race', 'age','education']]
mitigated_y = mitigated[['label']]

In [5]:
original_X_train, original_X_test, original_y_train, original_y_test = train_test_split(original_X, original_y, test_size=0.3, random_state=54)
mitigated_X_train, mitigated_X_test, mitigated_y_train, mitigated_y_test = train_test_split(mitigated_X, mitigated_y, test_size=0.3, random_state=54)

In [6]:
# fit model no training data
model_original = XGBClassifier()
model_original.fit(original_X, original_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [7]:
model_mitigated = XGBClassifier()
model_mitigated.fit(mitigated_X, mitigated_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [8]:
original_y['Predicted'] = model_original.predict(original_X)
mitigated_y['Predicted'] = model_mitigated.predict(mitigated_X)

/tmp/ipykernel_1026/679083925.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_y['Predicted'] = model_original.predict(original_X)
/tmp/ipykernel_1026/679083925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mitigated_y['Predicted'] = model_mitigated.predict(mitigated_X)


In [9]:
original_X[['label', 'Predicted']] = original_y[['label', 'Predicted']]
mitigated_X[['label', 'Predicted']] = mitigated_y[['label', 'Predicted']]

In [10]:
from FLAI import data
original_dataset = data.Data(original_X, transform=False)
mitigated_dataset = data.Data(mitigated_X, transform=False)

In [11]:
original_dataset.fairness_metrics(target_column='label', predicted_column = 'Predicted',
                            columns_fair = {'sex' : {'privileged' : 1, 'unprivileged' : 0},
                                           'race' : {'privileged' : 1, 'unprivileged' : 0}})

Calculating metrics for : sex  the value :  1
Calculating metrics for : sex  the value :  0
Calculating metrics for : race  the value :  1
Calculating metrics for : race  the value :  0


{'model': {'ACC': 0.8052297522594987,
  'TPR': 0.38996803540693387,
  'FPR': 0.06513145269621953,
  'FNR': 0.6100319645930662,
  'PPP': 0.14241422679809296},
 'sex': {'privileged': {'ACC': 0.7635894520488039,
   'TPR': 0.4609795087923267,
   'FPR': 0.10615538596271738,
   'FNR': 0.5390204912076733,
   'PPP': 0.21292692526348012},
  'unprivileged': {'ACC': 0.8893305069775658,
   'TPR': 0.0,
   'FPR': 0.0,
   'FNR': 1.0,
   'PPP': 0.0},
  'fair_metrics': {'Equal_Opportunity_Difference': -0.4609795087923267,
   'Disparate Impact': 0.0,
   'Statistical parity Difference': -0.21292692526348012}},
 'race': {'privileged': {'ACC': 0.7957932239388853,
   'TPR': 0.405625764370159,
   'FPR': 0.07269146207401979,
   'FNR': 0.594374235629841,
   'PPP': 0.1566236168682128},
  'unprivileged': {'ACC': 0.860344137655062,
   'TPR': 0.24129032258064517,
   'FPR': 0.02604783329386692,
   'FNR': 0.7587096774193548,
   'PPP': 0.05942376950780312},
  'fair_metrics': {'Equal_Opportunity_Difference': -0.164335

In [12]:
mitigated_dataset.fairness_metrics(target_column='label', predicted_column = 'Predicted',
                            columns_fair = {'sex' : {'privileged' : 1, 'unprivileged' : 0},
                                            'race' : {'privileged' : 1, 'unprivileged' : 0}})

Calculating metrics for : sex  the value :  1
Calculating metrics for : sex  the value :  0
Calculating metrics for : race  the value :  1
Calculating metrics for : race  the value :  0


{'model': {'ACC': 0.788996460849981,
  'TPR': 0.4424789708065314,
  'FPR': 0.1036966098448573,
  'FNR': 0.5575210291934686,
  'PPP': 0.18380180759893533},
 'sex': {'privileged': {'ACC': 0.7859216893972724,
   'TPR': 0.44251064617663394,
   'FPR': 0.1070459922672976,
   'FNR': 0.5574893538233661,
   'PPP': 0.18675758908930928},
  'unprivileged': {'ACC': 0.795095558076621,
   'TPR': 0.4424152068579948,
   'FPR': 0.09708295350957157,
   'FNR': 0.5575847931420053,
   'PPP': 0.1779387381097827},
  'fair_metrics': {'Equal_Opportunity_Difference': -9.543931863914557e-05,
   'Disparate Impact': 0.9527791560036185,
   'Statistical parity Difference': -0.008818850979526588}},
 'race': {'privileged': {'ACC': 0.7894085281980743,
   'TPR': 0.46430129416896904,
   'FPR': 0.10989505922623069,
   'FNR': 0.535698705831031,
   'PPP': 0.193707015130674},
  'unprivileged': {'ACC': 0.7866510080250538,
   'TPR': 0.31814415907207955,
   'FPR': 0.06842644797539724,
   'FNR': 0.6818558409279205,
   'PPP': 0.12

In [13]:
#https://transformernlp.medium.com/ai-fairness-a-brief-introduction-to-ai-fairness-360-b2e39c96ca49
#https://github.com/Trusted-AI/AIF360/blob/master/examples/README.md

In [14]:
original_y['Predicted_mitidata'] = model_mitigated.predict(original_X[['sex', 'race', 'age', 'education']])
original_X['Predicted_mitidata'] = original_y['Predicted_mitidata']


In [15]:
original_dataset.fairness_metrics(target_column='label', predicted_column = 'Predicted_mitidata',
                            columns_fair = {'sex' : {'privileged' : 1, 'unprivileged' : 0},
                                           'race' : {'privileged' : 1, 'unprivileged' : 0}})

Calculating metrics for : sex  the value :  1
Calculating metrics for : sex  the value :  0
Calculating metrics for : race  the value :  1
Calculating metrics for : race  the value :  0


{'model': {'ACC': 0.7905759162303665,
  'TPR': 0.4487337103516105,
  'FPR': 0.10270581462291307,
  'FNR': 0.5512662896483895,
  'PPP': 0.1850302728947907},
 'sex': {'privileged': {'ACC': 0.762846022652731,
   'TPR': 0.44775468681877634,
   'FPR': 0.10152633554360065,
   'FNR': 0.5522453131812236,
   'PPP': 0.20571128700747804},
  'unprivileged': {'ACC': 0.8465818759936407,
   'TPR': 0.45411013567438147,
   'FPR': 0.10457840897805144,
   'FNR': 0.5458898643256185,
   'PPP': 0.1432609079667903},
  'fair_metrics': {'Equal_Opportunity_Difference': 0.006355448855605128,
   'Disparate Impact': 0.6964173432135616,
   'Statistical parity Difference': -0.06245037904068773}},
 'race': {'privileged': {'ACC': 0.7817135418450892,
   'TPR': 0.4636499524391901,
   'FPR': 0.11107548552583364,
   'FNR': 0.5363500475608098,
   'PPP': 0.1999588914391422},
  'unprivileged': {'ACC': 0.8423369347739096,
   'TPR': 0.30709677419354836,
   'FPR': 0.059436419606914516,
   'FNR': 0.6929032258064516,
   'PPP': 0.